In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
import warnings

warnings.simplefilter("ignore")

In [2]:
# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Paths to your dataset
train_dir = 'dataset/train'  # structure: train/class_x/img.jpg
val_dir = 'dataset/val'

Using device: cuda


In [3]:
# Data augmentation & normalization for training
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  # Imagenet means
                         [0.229, 0.224, 0.225])  # Imagenet stds
])

# Just normalization for validation
val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [4]:
# Datasets & Loaders
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
val_dataset = datasets.ImageFolder(val_dir, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)


In [8]:
# Model setup
model = models.resnet50(pretrained=True)

num_classes = len(os.listdir('dataset/train'))

# Freeze earlier layers
for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes) 
model = model.to(device)

In [9]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Training loop with validation and tqdm progress bar
epochs = 10
best_val_acc = 0.0

In [10]:
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{epochs}] - Train", leave=False)
    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        loop.set_postfix(loss=loss.item(), accuracy=100 * correct / total)

    train_loss = running_loss / total
    train_acc = 100 * correct / total

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        val_loop = tqdm(val_loader, desc=f"Epoch [{epoch+1}/{epochs}] - Val", leave=False)
        for images, labels in val_loop:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

            val_loop.set_postfix(loss=loss.item(), accuracy=100 * val_correct / val_total)

    val_loss /= val_total
    val_acc = 100 * val_correct / val_total

    print(f"Epoch {epoch+1}/{epochs}: "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")


Epoch 1/10: Train Loss: 0.7591, Train Acc: 79.72%, Val Loss: 0.3262, Val Acc: 90.00%


Epoch 2/10: Train Loss: 0.4357, Train Acc: 86.69%, Val Loss: 0.2504, Val Acc: 92.29%


Epoch 3/10: Train Loss: 0.3810, Train Acc: 87.96%, Val Loss: 0.2530, Val Acc: 91.71%


Epoch 4/10: Train Loss: 0.3676, Train Acc: 88.50%, Val Loss: 0.2149, Val Acc: 93.17%


Epoch 5/10: Train Loss: 0.3511, Train Acc: 88.91%, Val Loss: 0.1956, Val Acc: 93.84%


Epoch 6/10: Train Loss: 0.3362, Train Acc: 89.58%, Val Loss: 0.1905, Val Acc: 93.67%


Epoch 7/10: Train Loss: 0.3211, Train Acc: 89.91%, Val Loss: 0.2025, Val Acc: 93.58%


Epoch 8/10: Train Loss: 0.3258, Train Acc: 89.88%, Val Loss: 0.2617, Val Acc: 92.25%


Epoch 9/10: Train Loss: 0.3221, Train Acc: 90.09%, Val Loss: 0.1437, Val Acc: 95.22%


Epoch 10/10: Train Loss: 0.3048, Train Acc: 90.50%, Val Loss: 0.2039, Val Acc: 93.51%


In [11]:
torch.save(model.state_dict(), 'model.pth')

In [12]:
from PIL import Image
from torchvision import transforms

# Define the transform (same as validation transform)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])
])

# Function to predict image class
def predict_image(img_path, model, class_names):
    img = Image.open(img_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(img_tensor)
        _, predicted = torch.max(outputs, 1)

    predicted_class = class_names[predicted.item()]
    print(f"Predicted class: {predicted_class}")
    return predicted_class

# Example usage:
class_names = train_dataset.classes  # Or manually: ['Tomato___Bacterial_spot', ...]
predict_image("./dataset/val/Tomato___Bacterial_spot/0a6d40e4-75d6-4659-8bc1-22f47cdb2ca8___GCREC_Bact.Sp 6247.JPG", model, class_names)


Predicted class: Tomato___Bacterial_spot


'Tomato___Bacterial_spot'

In [13]:
#  Using TorchServe to serve the model
model.eval()
example_input = torch.rand(1, 3, 224, 224).to(device)
traced_script_module = torch.jit.trace(model, example_input)
traced_script_module.save("Model.pt")



In [18]:
# getting all the class and their respective indexes in a json file
import json
idx_to_class = {v: k for k, v in train_dataset.class_to_idx.items()}

with open("index_to_name.json", "w") as f:
    json.dump(idx_to_class, f, indent=4)
